## Dissecting baseComcamLoop.py

### Motivation 

A notebook to understand how does `baseComcamLoop.py` work in detail, including all the dependencies on `ts_wep`, `ts_ofc`  and `ts_phosim`.  We need to be able to describe how the feedback loop is run, including all the creation of all  output directories  to make the `star_separation_test.ipynb`.

In [13]:
import os
import argparse
import numpy as np
import shutil

from lsst.ts.wep.ParamReader import ParamReader
from lsst.ts.wep.Utility import FilterType, CamType, runProgram
from lsst.ts.wep.ctrlIntf.WEPCalculationFactory import WEPCalculationFactory
from lsst.ts.wep.ctrlIntf.RawExpData import RawExpData

from lsst.ts.ofc.Utility import InstName
from lsst.ts.ofc.ctrlIntf.OFCCalculationFactory import OFCCalculationFactory

from lsst.ts.phosim.telescope.TeleFacade import TeleFacade
from lsst.ts.phosim.PhosimCmpt import PhosimCmpt
from lsst.ts.phosim.SkySim import SkySim
from lsst.ts.phosim.Utility import getPhoSimPath, getAoclcOutputPath, \
                                   getConfigDir
from lsst.ts.phosim.PlotUtil import plotFwhmOfIters


`baseComcamLoop.py` is run in the following manner inside `runStarSeparationAnalysis.py` : 
    

In [19]:
phosimDir = getPhoSimPath() 
outputDir = getAoclcOutputPath()
testLabel = 'sep'
skyFilePath  = 'starCat.txt'
#skyFilePath points to a phosim star catalog file : starCat.txt 

numPro = 8 # number of processors setting in phosimCmptSetting.yaml 
iterNum  = 1 # number of iterations 

starSep = 0.05 # star separation over which we are looping 
   
'''
    ccLoop.main(phosimDir, numPro, 1, outputDir, '%s.%.1f' % (testLabel, starSep), 
                    isEimg=False, genOpd=True, genDefocalImg=True, 
                    genFlats=True, useMinDofIdx=False,
                    inputSkyFilePath=skyFilePath, m1m3ForceError=0.05)
'''

"\n    ccLoop.main(phosimDir, numPro, 1, outputDir, '%s.%.1f' % (testLabel, starSep), \n                    isEimg=False, genOpd=True, genDefocalImg=True, \n                    genFlats=True, useMinDofIdx=False,\n                    inputSkyFilePath=skyFilePath, m1m3ForceError=0.05)\n"

The signature of that function is the following : 

In [20]:
'''class baseComcamLoop():

    def main(self, phosimDir, numPro, iterNum, baseOutputDir, 
            testName, isEimg=False, genOpd=True, genDefocalImg=True, genFlats=True,
            surveyFilter=None, starMag=15, 
            useMinDofIdx=False, inputSkyFilePath="", m1m3ForceError=0.05):
'''

'class baseComcamLoop():\n\n    def main(self, phosimDir, numPro, iterNum, baseOutputDir, \n            testName, isEimg=False, genOpd=True, genDefocalImg=True, genFlats=True,\n            surveyFilter=None, starMag=15, \n            useMinDofIdx=False, inputSkyFilePath="", m1m3ForceError=0.05):\n'

This means that the arguments passed to baseComcamLoop are : 
    


In [21]:
baseOutputDir = outputDir
testName = '%s.%.1f' % (testLabel, starSep)
isEimg=False; genOpd=True; genDefocalImg=True; genFlats=True; useMinDofIdx=False
surveyFilter=None
inputSkyFilePath=skyFilePath
m1m3ForceError=0.05

The beginning of baseComcamLoop main  :

In [24]:
# Prepare the calibration products (only for the amplifier images)
#sensorNameList = self._getComCamSensorNameList()
sensorNameList = ["R22_S00", "R22_S01", "R22_S02", "R22_S10", "R22_S11",
                        "R22_S12", "R22_S20", "R22_S21", "R22_S22"]


if ((not isEimg) & (genFlats is True)): # pass by default 
    #fakeFlatDir = self._makeCalibs(baseOutputDir, sensorNameList) : 
    # begin self._makeCalibs :
    outputDir = baseOutputDir
    fakeFlatDirName = "fake_flats"
    fakeFlatDir = os.path.join(outputDir, fakeFlatDirName)
    
    ## self._makeCalibs calls self._makeDir(fakeFlatDir) : 
    ## begin self._makeDir
    if (not os.path.exists(fakeFlatDir)):
            os.makedirs(fakeFlatDir)
    ## end self._makeDir
    
    # back to self._makeCalibs
    detector = " ".join(sensorNameList)
    #self._makeCalibs calls  self._genFakeFlat(fakeFlatDir, detector) : 
    
    ## begin self._genFakeFlat
    currWorkDir = os.getcwd()
    os.chdir(fakeFlatDir)
    ## self._genFakeFlat calls self._makeFakeFlat(detector)
    
    ### begin self._makeFakeFlat
    command = "makeGainImages.py"
    argstring = "--detector_list %s" % detector
    runProgram(command, argstring=argstring)
    ### end self._makeFakeFlat
    
    # back to self._genFakeFlat
    os.chdir(currWorkDir)

    # return fakeFlatDir
    # end self._makeCalibs
    
    
# Make the ISR directory
isrDirName = "input"
isrDir = os.path.join(baseOutputDir, isrDirName)
if genFlats is True:
    ## self.main  calls self._makeDir(isrDir) : 
    ## begin self._makeDir
    if (not os.path.exists(isrDir)):
            os.makedirs(isrDir)
    ## end self._makeDir

In [25]:
# Survey parameters
surveySettingFilePath = os.path.join(getConfigDir(),
                                    "surveySettings.yaml")
surveySettings = ParamReader(filePath=surveySettingFilePath)
if surveyFilter is None:
    filterType = FilterType.fromString(surveySettings.getSetting("filterType"))
else:
    filterType = FilterType.fromString(surveyFilter)
raInDeg = surveySettings.getSetting("raInDeg")
decInDeg = surveySettings.getSetting("decInDeg")
rotAngInDeg = surveySettings.getSetting("rotAngInDeg")

Now `self.main()` calls  `self._preparePhosimCmpt()`:

In [29]:
# Prepare the components
# self.main calls phosimCmpt = self._preparePhosimCmpt(phosimDir, filterType, raInDeg, decInDeg,
#                                rotAngInDeg, numPro, isEimg, m1m3ForceError)

# whose signature is _preparePhosimCmpt(self, phosimDir, filterType, raInDeg, decInDeg, rotAngInDeg,
#                       numPro, isEimg, m1m3ForceError)

# so the params passed are called the same way 

# begin self._preparePhosimCmpt:

# Set the Telescope facade class
tele = TeleFacade()
tele.addSubSys(addCam=True, addM1M3=True, addM2=True)
tele.setPhoSimDir(phosimDir)

In [30]:
# now the AnalysisPhoSimCmpt() is instantiated, so need to define it : 

class AnalysisPhosimCmpt(PhosimCmpt):

    def _writeOpdZkFile(self, zkFileName, rotOpdInDeg):
        """Write the OPD in zk file.
        OPD: optical path difference.
        Parameters
        ----------
        zkFileName : str
            OPD in zk file name.
        rotOpdInDeg : float
            Rotate OPD in degree in the counter-clockwise direction.
        """

        testOutputDir = os.environ["closeLoopTestDir"]
        filePath = os.path.join(testOutputDir, zkFileName)
        opdData = self._mapOpdToZk(rotOpdInDeg)
        header = "The followings are OPD in rotation angle of %.2f degree in um from z4 to z22:" % (
            rotOpdInDeg)
        np.savetxt(filePath, opdData, header=header)

    def reorderAndSaveWfErrFile(self, listOfWfErr, refSensorNameList,
                                zkFileName="wfs.zer"):
        """Reorder the wavefront error in the wavefront error list according to
        the reference sensor name list and save to a file.
        The unexisted wavefront error will be a numpy zero array. The unit is
        um.
        Parameters
        ----------
        listOfWfErr : list [lsst.ts.wep.ctrlIntf.SensorWavefrontData]
            List of SensorWavefrontData object.
        refSensorNameList : list
            Reference sensor name list.
        zkFileName : str, optional
            Wavefront error file name. (the default is "wfs.zer".)
        """

        # Get the sensor name that in the wavefront error map
        wfErrMap = self._transListOfWfErrToMap(listOfWfErr)
        nameListInWfErrMap = list(wfErrMap.keys())

        # Reorder the wavefront error map based on the reference sensor name
        # list.
        reorderedWfErrMap = dict()
        for sensorName in refSensorNameList:
            if sensorName in nameListInWfErrMap:
                wfErr = wfErrMap[sensorName]
            else:
                numOfZk = self.getNumOfZk()
                wfErr = np.zeros(numOfZk)
            reorderedWfErrMap[sensorName] = wfErr

        # Save the file
        testOutputDir = os.environ["closeLoopTestDir"]
        filePath = os.path.join(testOutputDir, zkFileName)
        wfsData = self._getWfErrValuesAndStackToMatrix(reorderedWfErrMap)
        header = "The followings are ZK in um from z4 to z22:"
        np.savetxt(filePath, wfsData, header=header)

        
# Prepare the phosim component
phosimCmpt = AnalysisPhosimCmpt(tele)

# Set the telescope survey parameters
boresight = (raInDeg, decInDeg)
zAngleInDeg = 27.0912
phosimCmpt.setSurveyParam(filterType=filterType, boresight=boresight,
                        zAngleInDeg=zAngleInDeg, rotAngInDeg=rotAngInDeg)

# Update the setting file if needed
settingFile = phosimCmpt.getSettingFile()
if (numPro > 1):
    settingFile.updateSetting("numPro", numPro)
if isEimg:
    settingFile.updateSetting("e2ADC", 0)

# Set the seed number for M1M3 surface
seedNum = 6
phosimCmpt.setSeedNum(seedNum)

# Set the M1M3 force error
phosimCmpt.setM1M3ForceError(m1m3ForceError)




Back to `self.main()` : 
    
    




In [31]:
# self.main() calls  wepCalc = self._prepareWepCalc(isrDir, filterType, raInDeg, decInDeg,
#                        rotAngInDeg, isEimg)

# whose signature is 
# _prepareWepCalc(self, isrDirPath, filterType, raInDeg, decInDeg, rotAngInDeg,
#                        isEimg)

# so the params are passed without name change apart from :
isrDirPath = isrDir

# begin self._prepareWepCalc()

wepCalc = WEPCalculationFactory.getCalculator(CamType.ComCam, isrDirPath)
wepCalc.setFilter(filterType)
wepCalc.setBoresight(raInDeg, decInDeg)
wepCalc.setRotAng(rotAngInDeg)

if (isEimg):
    settingFile = wepCalc.getSettingFile()
    settingFile.updateSetting("imageType", "eimage")
            
# end self._prepareWepCalc()

# back to self.main() 
tele = phosimCmpt.getTele()
defocalDisInMm = tele.getDefocalDistInMm()
wepCalc.setDefocalDisInMm(defocalDisInMm)

# self.main() calls 
# ofcCalc = self._prepareOfcCalc(filterType, rotAngInDeg)
# begin self._prepareOfcCalc()
ofcCalc = OFCCalculationFactory.getCalculator(InstName.COMCAM)
ofcCalc.setFilter(filterType)
ofcCalc.setRotAng(rotAngInDeg)
ofcCalc.setGainByPSSN()
# end self._prepareOfcCalc()

In [32]:
# Ingest the calibration products (only for the amplifier images)
if ((not isEimg) & (genFlats is True)):
    wepCalc.ingestCalibs(fakeFlatDir)

# Only use 10 hexapod positions and first 3 bending modes of M1M3 and M2
if (useMinDofIdx): # this is false 
    self._useMinDofIdx(ofcCalc)

# Set the telescope state to be the same as the OFC
state0 = ofcCalc.getStateAggregated()
phosimCmpt.setDofInUm(state0)

In [33]:
# Do the iteration
obsId = 9006000
opdZkFileName = str("opd.zer" + '.' + testName)
wfsZkFileName = str("wfs.zer" + '.' + testName)
opdPssnFileName = "PSSN.txt"
outputDirName = "pert"
outputImgDirName = "img"
iterDefaultDirName = "iter"
dofInUmFileName = "dofPertInNextIter.mat"
skyInfoFileName = "skyComCamInfo.txt"



In [34]:
opdZkFileName


'opd.zer.sep.0.1'

In [35]:
wfsZkFileName

'wfs.zer.sep.0.1'

In [36]:
iterNum = 1 
#for iterCount in range(iterNum):

#### beginning of iteration 

# Set the observation Id
phosimCmpt.setSurveyParam(obsId=obsId)

# The iteration directory
iterDirName = "%s%d" % (iterDefaultDirName, iterCount)

# Set the output directory
outputDir = os.path.join(baseOutputDir, iterDirName, outputDirName)
phosimCmpt.setOutputDir(outputDir)

# Set the output image directory
outputImgDir = os.path.join(baseOutputDir, iterDirName,
                            outputImgDirName)
phosimCmpt.setOutputImgDir(outputImgDir)




In [37]:
# Generate the OPD image
if genOpd is True:
    argString = phosimCmpt.getComCamOpdArgsAndFilesForPhoSim()
    phosimCmpt.runPhoSim(argString)
    
    

In [44]:
os.path.realpath(__file__)

NameError: name '__file__' is not defined

In [45]:
testOutput = outputDir # just for now .. 
if (testOutput == ""):
    testOutputDir = os.path.dirname(os.path.realpath(__file__))
else:
    testOutputDir = testOutput


os.environ["closeLoopTestDir"] = testOutputDir

# Analyze the OPD data
phosimCmpt.analyzeComCamOpdData(zkFileName=opdZkFileName,
                                pssnFileName=opdPssnFileName)

In [46]:
# Get the PSSN from file
pssn = phosimCmpt.getOpdPssnFromFile(opdPssnFileName)
print("Calculated PSSN is %s." % pssn)

# Get the GQ effective FWHM from file
gqEffFwhm = phosimCmpt.getOpdGqEffFwhmFromFile(opdPssnFileName)
print("GQ effective FWHM is %.4f." % gqEffFwhm)

# Set the FWHM data
listOfFWHMSensorData = phosimCmpt.getListOfFwhmSensorData(
    opdPssnFileName, sensorNameList)
ofcCalc.setFWHMSensorDataOfCam(listOfFWHMSensorData)

Calculated PSSN is [0.59515269 0.59006253 0.59086933 0.59333341 0.58535632 0.58658711
 0.59610244 0.58819379 0.58696264].
GQ effective FWHM is 0.5429.


In [47]:
# Prepare the faked sky - this is skipped because inputSkyFilePath = 'starCat.txt'
if (inputSkyFilePath == ""):
    # According to the OPD field positions
    metr = phosimCmpt.getOpdMetr()
    skySim = self._prepareSkySim(metr, starMag)
    print("Use the default OPD field positions to be star positions.")
    print("The star magnitude is chosen to be %.2f." % starMag)
else:
    # self.main() calls skySim = self._prepareSkySimBySkyFile(inputSkyFilePath)
    # begin self._prepareSkySimBySkyFile
    skySim = SkySim()
    absSkyFilePath = os.path.abspath(inputSkyFilePath)
    skySim.addStarByFile(absSkyFilePath)
    # end self._prepareSkySimBySkyFile


In [48]:
# Output the sky information
# self.main() calls skySim, wepCalc = self._outputSkyInfo(outputDir, skyInfoFileName, skySim, wepCalc)

# begin self._outputSkyInfo
outputSkyInfoFilePath = os.path.join(outputDir, skyInfoFileName)
skySim.exportSkyToFile(outputSkyInfoFilePath)
wepCalc.setSkyFile(outputSkyInfoFilePath)
# end self._outputSkyInfo 


# Assign the entra- and intra-focal observation Id
extraObsId = obsId + 1
intraObsId = obsId + 2

In [50]:
# Generate the defocal images
simSeed = 1000
argStringList = phosimCmpt.getComCamStarArgsAndFilesForPhoSim(extraObsId, intraObsId, skySim, simSeed=simSeed,
    cmdSettingFileName="starDefault.cmd", instSettingFileName="starSingleExp.inst")
if genDefocalImg is True:
    for argString in argStringList:
        phosimCmpt.runPhoSim(argString)

    # Repackage the images based on the image type
    if (isEimg):
        phosimCmpt.repackageComCamEimgFromPhoSim()
    else:
        phosimCmpt.repackageComCamAmpImgFromPhoSim()

In [51]:
# Collect the defocal images
intraRawExpData = RawExpData()
intraRawExpDir = os.path.join(outputImgDir,
                            phosimCmpt.getIntraFocalDirName())
intraRawExpData.append(intraObsId, 0, intraRawExpDir)

extraRawExpData = RawExpData()
extraRawExpDir = os.path.join(outputImgDir,
                            phosimCmpt.getExtraFocalDirName())
extraRawExpData.append(extraObsId, 0, extraRawExpDir)


The error I encountered last time pertains to this line  : l.237 in `baseComcamLoop.py` .... 

File "runStarSeparationAnalysis.py", line 77, in <module>
    inputSkyFilePath=skyFilePath, m1m3ForceError=0.05)
  File "/data/epyc/users/suberlak/Commissioning/aos/ts_phosim/notebooks/analysis_scripts/baseComcamLoop.py", line 237, in main
    intraRawExpData, extraRawExpData=extraRawExpData)
  File "/astro/store/epyc/projects/lsst_comm/ts_wep/python/lsst/ts/wep/ctrlIntf/WEPCalculation.py", line 391, in calculateWavefrontErrors
    donutMap = self._calcWfErr(neighborStarMap, obsIdList)
  File "/astro/store/epyc/projects/lsst_comm/ts_wep/python/lsst/ts/wep/ctrlIntf/WEPCalculation.py", line 598, in _calcWfErr
    doDeblending=doDeblending)
  File "/astro/store/epyc/projects/lsst_comm/ts_wep/python/lsst/ts/wep/WepController.py", line 371, in getDonutMap
    np.savetxt('/astro/store/epyc/projects/lsst_comm/ts_phosim/notebooks/analysis_scripts/example.out', singleSciNeiImg)
  File "/astro/store/epyc/projects/lsst_comm/new_stack/python/miniconda3-4.7.10/envs/lsst-scipipe-4d7b902/lib/python3.7/site-packages/numpy/lib/npyio.py", line 1352, in savetxt
    open(fname, 'wt').close()
PermissionError: [Errno 13] Permission denied: '/astro/store/epyc/projects/lsst_comm/ts_phosim/notebooks/analysis_scripts/example.out'

In [53]:
listOfWfErr

[]

In [56]:
intraRawExpData

In [57]:
extraRawExpData

In [52]:
# Calculate the wavefront error and DOF
listOfWfErr = wepCalc.calculateWavefrontErrors(intraRawExpData, extraRawExpData=extraRawExpData)
ofcCalc.calculateCorrections(listOfWfErr)

IndexError: list index out of range

In [ ]:
# Record the wfs error with the same order as OPD for the comparison
phosimCmpt.reorderAndSaveWfErrFile(listOfWfErr, sensorNameList,
                                   zkFileName=wfsZkFileName)

# Set the new aggregated DOF to phosimCmpt
dofInUm = ofcCalc.getStateAggregated()
phosimCmpt.setDofInUm(dofInUm)

In [ ]:
# Save the DOF file
phosimCmpt.saveDofInUmFileForNextIter(dofInUm, dofInUmFileName=dofInUmFileName)

# Add the observation ID by 10 for the next iteration
obsId += 10

#### end of iteration 

In [ ]:
# Summarize the FWHM
pssnFiles = [os.path.join(baseOutputDir, "%s%d" % (iterDefaultDirName, num),
            outputImgDirName, opdPssnFileName) for num in range(iterNum)]
saveToFilePath = os.path.join(baseOutputDir, "fwhmIters.png")
plotFwhmOfIters(pssnFiles, saveToFilePath=saveToFilePath)